In [45]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [46]:
df_1 = pd.read_csv("INFO_PELIS/cortos_2000_2006.csv", index_col = 0)
df_2 = pd.read_csv("INFO_PELIS/cortos_2021.csv", index_col = 0)
df_3 = pd.read_csv("INFO_PELIS/cortos_2022.csv", index_col = 0)
df_4 = pd.read_csv("INFO_PELIS/cortos_2023.csv", index_col = 0)
df_5 = pd.read_csv("INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)
df_6 = pd.read_csv("INFO_PELIS/pelis_2000_2006.csv", index_col = 0)
df_7 = pd.read_csv("INFO_PELIS/pelis_2007_2010.csv", index_col = 0)
df_8 = pd.read_csv("INFO_PELIS/pelis_2011_2013.csv", index_col = 0)
df_9 = pd.read_csv("INFO_PELIS/cortos_2007_2008.csv", index_col = 0)
df_10 = pd.read_csv("INFO_PELIS/cortos_2009_2010.csv", index_col = 0)
df_11 = pd.read_csv("INFO_PELIS/cortos_2011.csv", index_col = 0)
df_12 = pd.read_csv("INFO_PELIS/cortos_2012.csv", index_col = 0)
df_13 = pd.read_csv("INFO_PELIS/cortos_2013.csv", index_col = 0)



In [47]:
df_peliculas = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13], axis=0)

In [48]:
df_peliculas.to_csv('union_peliculas.csv')

In [49]:
df_peliculas.head()

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID,Género
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874,NaN
1,Drama,Short,Nights,2000,3.0,tt0138655,NaN
2,Drama,Short,Caught,2000,8.0,tt0142155,NaN
3,Drama,Short,Smog,2000,3.0,tt0144613,NaN
4,Drama,Short,Safe Journey,2000,NaN,tt0188189,NaN


In [50]:
df_peliculas.tail()

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID,Género
29062,NaN,Short,Snake Eater,2013,7.0,tt28769878,Action
29063,NaN,Short,Boom Box,2013,10.0,tt28970681,Action
29064,NaN,Short,Maudit Poulet,2013,NaN,tt29279117,Action
29065,NaN,Short,I___U,2013,8.0,tt29433024,Action
29066,NaN,Short,Blind-Sided,2013,12.0,tt30310933,Action


In [51]:
lista_titulos= df_peliculas["Título"].to_list()

In [52]:
lista_ID = df_peliculas["ID"].to_list()

In [78]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}

lista_ID

for pelicula in lista_ID[130:133]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    
# Seleccionar nombre de la pelicula
    try:
        diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text)
    except:
        diccionario_peliculas["Nombre de la película"].append(f"Error nombre para la pelicula con id: {pelicula}")
        
# Seleccionar puntuacion 
    try: 
        diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text)
        
    except:  
        diccionario_peliculas["Puntuacion_IMDB"].append("--")

# Seleccionar director
    try:          
        diccionario_peliculas["Dirección"].append([e.text for e in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]:first-child > div > ul > li > a') if e.text != ""])
    except:
        diccionario_peliculas["Dirección"].append("DESCONOCIDO") 

# Seleccionar guionistas        
    try:
        diccionario_peliculas["Guionistas"].append([e.text for e in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]:nth-child(2) > div > ul > li > a') if e.text != ""])
    except:
        diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
    
    sleep(5)
# Seleccionar ARGUMENTO
    
    try:     
        
        diccionario_peliculas["Argumento"].append(driver.find_element("css selector", 'span[data-testid="plot-xl"]').text)
            
    except:
     
        diccionario_peliculas["Argumento"].append("Desconocido")

# Seleccionar DURACION        
    try:   
        busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
        if "min" in busqueda_duracion or "h" in busqueda_duracion:
            diccionario_peliculas["Duración"].append(busqueda_duracion)
        else:
            diccionario_peliculas["Duración"].append("--")
    except:
            diccionario_peliculas["Duración"].append("--")



### REVISAR:
# Lista de tuplas
# Orden de guionista/actores


In [79]:
diccionario_peliculas

{'Puntuacion_IMDB': ['7,6', '--', '--'],
 'Dirección': [['Kalman Apple'], ['Heather Dyer'], ['Stephen Winter']],
 'Guionistas': [['Kalman Apple', 'Anton Chekhov'],
  ['Heather Dyer'],
  ['Bryan Webster', 'Ron Butler']],
 'Argumento': ["Three actors and a director perform Chekhov's The Bear on a New York City bus (actually, a succession of buses, as they keep getting thrown off). The other riders on the final bus get progressively more interested.",
  '',
  ''],
 'Duración': ['28min', '--', '3min'],
 'Nombre de la película': ['Speed for Thespians', 'The Valentine', 'Untitled']}

In [63]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(5)

## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Rotten Tomatoes":[]}

lista_titulos

for pelicula in lista_titulos[:10]:
    
    busqueda = f"puntuacion Rotten Tomatoes de la {pelicula}"


    sleep(5)


    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)


    sleep(5)
    try:
        diccionario_peliculas["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
    except:
        puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
        puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
        diccionario_peliculas["Rotten Tomatoes"].append(puntuacion.group())




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#rso > div:nth-child(1) > div > div > div:nth-child(3) > div"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF63BFD7012+3522402]
	(No symbol) [0x00007FF63BBF8352]
	(No symbol) [0x00007FF63BAA5ABB]
	(No symbol) [0x00007FF63BAEBF0E]
	(No symbol) [0x00007FF63BAEC08C]
	(No symbol) [0x00007FF63BB2E437]
	(No symbol) [0x00007FF63BB0F09F]
	(No symbol) [0x00007FF63BB2BDA3]
	(No symbol) [0x00007FF63BB0EE03]
	(No symbol) [0x00007FF63BADF4D4]
	(No symbol) [0x00007FF63BAE05F1]
	GetHandleVerifier [0x00007FF63C009B9D+3730157]
	GetHandleVerifier [0x00007FF63C05F02D+4079485]
	GetHandleVerifier [0x00007FF63C0575D3+4048163]
	GetHandleVerifier [0x00007FF63BD2A649+718233]
	(No symbol) [0x00007FF63BC04A3F]
	(No symbol) 

In [62]:
print(diccionario_peliculas)

{'Rotten Tomatoes': ['80%']}
